In [ ]:
%matplotlib widget

# SANS data reduction

This notebook will guide you through the data reduction for the SANS experiment that you simulated with McStas yesterday.

The following is a basic outline of what this notebook will cover:

- Loading the NeXus files that contain the data
- Inspect/visualize the data contents
- How to convert the raw `time-of-flight` coordinate to something more useful ($\lambda$, $Q$, ...)
- Normalize to a flat field run
- Write the results to file

In [ ]:
import numpy as np
import scipp as sc
import plopp as pp
import sans_utils as utils

## Process the run with a sample

### Load the NeXus file data

In [ ]:
folder = "../3-mcstas/SANS_with_sample_many_neutrons"

⚠️ **If you did not complete the SANS with sample simulation yesterday**,
you can use some pre-prepared data by uncommenting and running the cell below:

In [ ]:
# folder = utils.fetch_data("3-mcstas/SANS_with_sample_many_neutrons")

In [ ]:
sample = utils.load_sans(folder)

The first way to inspect the data is to view the HTML representation of the loaded object.

Try to explore what is inside the data, and familiarize yourself with the different sections (`Dimensions`, `Coordinates`, `Data`).

In [ ]:
sample

### Visualize the data

Here is a 2D visualization of the neutron counts, histogrammed along the `tof` and `y` dimensions:

In [ ]:
sample.hist(tof=200, y=200).plot(norm="log", vmin=1.0e-2)

Histogramming along `y` only gives a 1D plot:

In [ ]:
sample.hist(y=200).plot(norm="log")

### Coordinate transformations

The first step in the data reduction is to convert the raw event coordinates (position, time-of-flight) to something physically meaningful such as wavelength ($\lambda$) or momentum transfer ($Q$).

Scipp has a dedicated method for this called `transform_coords` (see docs [here](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)).

We begin with a standard graph which describes how to compute e.g. the wavelength from the other coordinates in the raw data.

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import kinematic

graph = {**beamline(scatter=True), **kinematic("tof")}
sc.show_graph(graph, simplified=True)

To compute the wavelength of all the events, we simply call `transform_coords` on our loaded data,
requesting the name of the coordinate we want in the output (`"wavelength"`),
as well as providing it the graph to be used to compute it (i.e. the one we defined just above).

This yields

In [ ]:
sample_wav = sample.transform_coords("wavelength", graph=graph)
sample_wav

The result has a `wavelength` coordinate. We can also plot the result:

In [ ]:
sample_wav.hist(wavelength=200).plot()

We can see that the range of observed wavelengths agrees with the range set in the McStas model (5.25 - 6.75 Å)

### Exercise 1: convert raw data to $Q$

Instead of wavelength as in the example above,
the task is now to convert the raw data to momentum-transfer $Q$.

The transformation graph is missing the computation for $Q$ so you will have to add it in yourself.
As a reminder, $Q$ is computed as follows

$$Q = \frac{4\pi \sin(\theta)}{\lambda}$$

You have to:

- create a function that computes $Q$
- add it to the graph
- call `transform_coords` using the new graph

Note that the graph already contains the necessary components to compute the scattering angle $2 \theta$ (called `two_theta` in code).

**Solution:**

In [ ]:
# Insert your solution:


### Histogram the data in $Q$

The final step in processing the sample run is to histogram the data into $Q$ bins.

In [ ]:
sample_h = sample_q.hist(Q=200)
sample_h.plot(norm="log", vmin=1)

## Exercise 2: process flat-field run

Repeat the step carried out above for the run that contained no sample (also known as a "flat-field" run).

In [ ]:
folder = "../3-mcstas/SANS_without_sample_many_neutrons"

⚠️ **If you did not complete the SANS without sample simulation yesterday**,
you can use some pre-prepared data by uncommenting and running the cell below:

In [ ]:
# folder = utils.fetch_data("3-mcstas/SANS_without_sample_many_neutrons")

**Solution:**

In [ ]:
# Insert your solution:


**Bonus question:** can you explain why the counts in the flat-field data drop at high $Q$?

## Exercise 3: Normalize the sample run

The flat-field run is giving a measure of the efficiency of each detector pixel.
This efficiency now needs to be used to correct the counts in the sample run to yield a realistic $I(Q)$ profile.

In particular, this should remove any unwanted artifacts in the data,
such as the drop in counts around 0.03 Å<sup>-1</sup> due to the air bubble inside the detector tube.

Normalizing is essentially just dividing the sample run by the flat field run.

**Hint:** you may run into an error like `"Mismatch in coordinate 'Q'"`. Why is this happening? How can you get around it?

**Solution:**

In [ ]:
# Insert your solution:


In [ ]:
# Insert your solution:


In [ ]:
normed.plot(norm="log", vmin=1.0e-3, vmax=10.0)

## Save result to disk

Finally, we need to save our results to disk,
so that the reduced data can be forwarded to the next step in the pipeline (data analysis).

We will use a simple text file for this:

In [ ]:
from scippneutron.io import save_xye

# The simple file format does not support bin-edge coordinates.
# So we convert to bin-centers first.
data = normed.copy()
data.coords["Q"] = sc.midpoints(data.coords["Q"])

save_xye("sans_iofq.dat", data)

## Bonus exercise

Re-run the reduction using the results from the simulations with less neutrons,
and compare the results.

**Solution:**

In [ ]:
# Insert your solution:


## Sciline workflow

<div class="alert alert-warning">

**Pause here if you have not yet been introduced to Sciline!**

</div>

Below we build a Sciline workflow for reducing the SANS data.
We wrap the operations that were carried out throughout this notebook into function that have the correct type-annotations,
building a pipeline of connected steps.

The custom types have been created for you in the `sans_utils` module, and are imported from there.

Take some time to read through the code and understand the different parts.
Ask questions if anything is unclear.

In [ ]:
import sciline as sl
from sans_utils import *


def load(folder: Foldername[RunType]) -> RawData[RunType]:
    """Load raw data from file"""
    return RawData[RunType](utils.load_sans(folder))


def to_wavelength(
    data: RawData[RunType], graph: CoordTransformGraph
) -> WavelengthData[RunType]:
    """Compute wavelength for events"""
    return WavelengthData[RunType](data.transform_coords("wavelength", graph=graph))


def to_Q(data: WavelengthData[RunType], graph: CoordTransformGraph) -> QData[RunType]:
    """Compute Q for events"""
    return QData[RunType](data.transform_coords("Q", graph=graph))


def to_histogram(events: QData[RunType], qbins: QBins) -> QHistogram[RunType]:
    """Histogram data in Q bins"""
    return QHistogram[RunType](events.hist(Q=qbins))


def normalize(
    sample: QHistogram[SampleRun], flat: QHistogram[FlatFieldRun]
) -> NormalizedQ:
    """Normalized sample by flat-field"""
    return NormalizedQ(sample / flat)


wf = sl.Pipeline(
    (load, to_wavelength, to_Q, to_histogram, normalize),
    constraints={RunType: [SampleRun, FlatFieldRun]},
)

wf.visualize()

### Exercise 4: Set the workflow parameters

In the visualization above, the red boxes indicate missing values;
parameters required by the computation that have yet to be set on the workflow.

Using the `wf[TYPE] = VALUE` syntax, set the missing parameters on the workflow.

Once you have done so, use `wf.compute(NormalizedQ)` to compute the final normalized result.

**Solution:**

In [ ]:
# Insert your solution:


### Exercise 5: Make a $(2\theta, Q)$ map

We wish to make a 2d plot of counts as a function of $2\theta$ and $Q$ for the `SampleRun`.
You have to:

- identify one of the intermediate results that would contain the necessary information
- compute that intermediate result
- histogram the result (200 bins in `two_theta` and 200 bins in `Q` will do nicely)
- plot it on a figure

**Hint:** Using `norm='log', vmin=1.0e-4` when plotting makes for a better-looking figure.

**Solution:**

In [ ]:
# Insert your solution:
